In [2]:
import pandas as pd;
import datetime as dt;
import numpy as np;
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import scale
import pickle

In [3]:
weather = pd.read_csv("../Data/weather (table 7)_training_update.csv")
volume = pd.read_csv("../Data/volume(table 6)_training.csv")

# volume
volume['time'] =  pd.to_datetime(volume['time'], format = "%Y-%m-%d %H:%M:%S")

time = []

for c in volume['time']:
    time.append(c - dt.timedelta(seconds = c.second, minutes = c.minute))
    
volume['time'] = time

volume = volume\
    .groupby(['time', 'tollgate_id', 'direction'])\
    .size()\
    .reset_index()\
    .rename(columns={0:'count'})

date = []
week = []
hour = []

for t in volume['time']:
    date.append(t - dt.timedelta(hours = t.hour))
    week.append(t.weekday())
    hour.append(t.hour)

volume['date'] = date
volume['week'] = week
volume['hour'] = hour

time_weather = []

for c in volume['time']:
    discard = dt.timedelta(hours = c.hour % 3, minutes = c.minute, seconds = c.second)
    if (discard > dt.timedelta(hours = 1, minutes = 30)):
        c = c + dt.timedelta(hours = 3)
    time_weather.append(c - discard)
    
volume['time_weather'] = time_weather

# weather
weather['time_weather'] = pd.to_timedelta(weather['hour'], unit="H") + pd.to_datetime(weather['date'], format="%Y-%m-%d")
del weather['date']
del weather['hour']

#merge
training = pd.merge(volume, weather, on = 'time_weather')

#drop
#914 - 918
droped = training[((training['date'] >= dt.date(year = 2016, day = 14, month = 9)) & \
              (training['date'] <= dt.date(year = 2016, day = 18, month = 9)))]

training = training.drop(droped.index)

#930 - 1007
droped = training[((training['date'] >= dt.date(year = 2016, day = 30, month = 9)) & \
              (training['date'] <= dt.date(year = 2016, day = 7, month = 10)))]

training = training.drop(droped.index)

#delete columns not need

del training['time_weather']
del training['date']

#re cols
training = training[list(training.columns.drop('count')) + ['count']]

#output csv
training.to_csv('training2.csv', index=False)